In [1]:
import pandas as pd
from typing import Literal
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from utils.loader import MovieLensDataset
from arquitecture.Recommender import Recommender
from arquitecture.components.PatterAnalyzer import PatternAnalyzer
import torch.optim as optim

SEED = 55
BATCH = 2000
NUN_THREADS = 6
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCH = 400
LEARNING_RATE = 0.0005


In [2]:
train_dataset = MovieLensDataset(ml_path="ml-100k", split="train", seed=SEED)
test_dataset = MovieLensDataset(ml_path="ml-100k", split="test", seed=SEED)
val_dataset = MovieLensDataset(ml_path="ml-100k", split="val", seed=SEED)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH, shuffle=True, num_workers=NUN_THREADS)

In [3]:
height=19
width=22
user_input_size = 23
    # Crear instancia del modelo
pattern_analyzer = PatternAnalyzer(conv_structure=[1,8,8,16,16,32,32],
                            input_size=torch.Size((height, width)),
                            pool_depth=3,
                            expert_hidden_size=12,
                            expert_output_len=8,
                            final_mlp_factor=2,
                            final_mlp_output_len=12,
                            ).to(DEVICE)
    
model = Recommender(
        user_data_input_size=user_input_size,
        user_data_analizer_factor = 1,
        user_data_analizer_output_size = 10,
        pattern_analyzer=pattern_analyzer,
        final_regressor_factor=1,
        final_regressor_output_len=19
    ).to(DEVICE)

print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

Model parameters: 513534


In [4]:
# Crear el optimizador, por ejemplo, usando Adam con una tasa de aprendizaje de 0.001
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Crear la función de pérdida para regresión (Mean Squared Error Loss)
criterion = nn.MSELoss(reduction="sum")

In [5]:
for epoch in range(NUM_EPOCH):
    # --- Training Phase ---
    model.train()  # Set model to training mode
    running_train_loss = 0.0
    
    for user_data_tensor, rating_train_tensor, rating_test_tensor in train_dataloader:
        optimizer.zero_grad()

        outputs = model(user_data_tensor.to(DEVICE), rating_train_tensor.to(DEVICE)).to(DEVICE)        
        loss = criterion(outputs.to(DEVICE), rating_test_tensor.to(DEVICE)) 
        
        loss.backward()                 # Backpropagation
        optimizer.step()                # Update model parameters
        running_train_loss += loss.item() * rating_test_tensor.size()[1]
    
    epoch_train_loss = running_train_loss / len(train_dataloader.dataset)

    # --- Validation Phase ---32
    model.eval()  # Set model to evaluation mode
    running_val_loss = 0.0
    with torch.no_grad():
        for user_data_tensor, rating_train_tensor, rating_test_tensor in val_dataloader:
            
            outputs = model(user_data_tensor.to(DEVICE), rating_train_tensor.to(DEVICE))        
            loss = criterion(outputs, rating_test_tensor.to(DEVICE)) 
            
            running_val_loss += loss.item() * rating_test_tensor.size()[1]
    
    epoch_val_loss = running_val_loss / len(val_dataloader.dataset)
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCH}] - Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")

Epoch [1/400] - Train Loss: 38.3259, Val Loss: 38.8306
Epoch [2/400] - Train Loss: 38.1760, Val Loss: 38.6755
Epoch [3/400] - Train Loss: 38.0289, Val Loss: 38.5218
Epoch [4/400] - Train Loss: 37.8833, Val Loss: 38.3681
Epoch [5/400] - Train Loss: 37.7371, Val Loss: 38.2165
Epoch [6/400] - Train Loss: 37.5917, Val Loss: 38.0724
Epoch [7/400] - Train Loss: 37.4530, Val Loss: 37.9336
Epoch [8/400] - Train Loss: 37.3198, Val Loss: 37.7947
Epoch [9/400] - Train Loss: 37.1868, Val Loss: 37.6558
Epoch [10/400] - Train Loss: 37.0540, Val Loss: 37.5132
Epoch [11/400] - Train Loss: 36.9176, Val Loss: 37.3661
Epoch [12/400] - Train Loss: 36.7770, Val Loss: 37.2177
Epoch [13/400] - Train Loss: 36.6353, Val Loss: 37.0679
Epoch [14/400] - Train Loss: 36.4924, Val Loss: 36.9132
Epoch [15/400] - Train Loss: 36.3448, Val Loss: 36.7525
Epoch [16/400] - Train Loss: 36.1917, Val Loss: 36.5822
Epoch [17/400] - Train Loss: 36.0295, Val Loss: 36.3967
Epoch [18/400] - Train Loss: 35.8525, Val Loss: 36.1935
E

KeyboardInterrupt: 